In [19]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from tqdm import tqdm

file_path = "./my-500-data.csv"
df = pd.read_csv(file_path, delimiter=';')
print(df.head())
print(df.columns)
df = df.iloc[:, 0].str.split('\t', expand=True)
df.columns = ['question', 'answer', 'final_QT', 'final_AS']
df = df[['question', 'final_QT']]

                question\tanswer\tfinal_QT\tfinal_AS
0  هل يعتبر الخوف من عدم الإنجاب مستقبلاً حالة عا...
1  من سنه تقريبا و انا أذي نفسي ب اكثر من طريقة و...
2  السلام عليكم مشكلتي تقتصر على تكرار كلمة معينة...
3  اكتئاب وفوبيا من المجتمع وانعزال وانطوائية وتع...
4  عمري 37 سنة وحامل في الشهر الثامن . منذ الشهر ...
Index(['question\tanswer\tfinal_QT\tfinal_AS'], dtype='object')


In [20]:
df['final_QT'] = df['final_QT'].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [21]:
df.head()

,question,final_QT
0,هل يعتبر الخوف من عدم الإنجاب مستقبلاً حالة عا...,"[A, D]"
1,من سنه تقريبا و انا أذي نفسي ب اكثر من طريقة و...,"[B, E]"
2,السلام عليكم مشكلتي تقتصر على تكرار كلمة معينة...,"[A, E]"
3,اكتئاب وفوبيا من المجتمع وانعزال وانطوائية وتع...,"[B, D]"
4,عمري 37 سنة وحامل في الشهر الثامن . منذ الشهر ...,"[A, B, D]"


In [22]:
from sklearn.preprocessing import MultiLabelBinarizer

# Encode labels into binary format
mlb = MultiLabelBinarizer()
labels_encoded = mlb.fit_transform(df['final_QT'])
labels_encoded

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [23]:
labels=mlb.classes_
labels

array(['A', 'B', 'C', 'D', 'E', 'F', 'Z'], dtype=object)

# Data Preprocessing

In [6]:
!pip install datasets evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.8 MB/s eta 0:00:00


In [24]:
from datasets import load_dataset,Dataset

In [25]:
ds=Dataset.from_pandas(df)

In [26]:
ds

Dataset({
    features: ['question', 'final_QT'],
    num_rows: 499
})

In [27]:
splited_ds=ds.train_test_split(test_size=0.2,seed=42)
splited_ds

DatasetDict({
    train: Dataset({
        features: ['question', 'final_QT'],
        num_rows: 399
    })
    test: Dataset({
        features: ['question', 'final_QT'],
        num_rows: 100
    })
})

In [45]:
train_ds, test_ds = splited_ds["train"], splited_ds["test"]
len(train_ds),len(test_ds)

(399, 100)

In [61]:
# load_tokenizer
model_name="NAMAA-Space/AraModernBert-Base-V1.0"
tokenizer=AutoTokenizer.from_pretrained(model_name)
MAX_LENGTH=128

tokenizer_config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.88M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [62]:
def tokenize(rows):
  return tokenizer(rows["question"],padding="max_length",truncation=True,max_length=MAX_LENGTH)

In [63]:
def label2onehot(rows):
  result_labels=[]
  for label in rows["final_QT"]:
    l=label
    result_labels.append([float(1) if x in l else float(0) for x in labels])
  return {"labels":result_labels}

In [64]:
train_ds=train_ds.map(tokenize,batched=True)
train_ds=train_ds.map(label2onehot,batched=True)
train_ds

Map:   0%|          | 0/399 [00:00<?, ? examples/s]

Map:   0%|          | 0/399 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'final_QT', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 399
})

In [65]:
test_ds=test_ds.map(tokenize,batched=True)
test_ds=test_ds.map(label2onehot,batched=True)
test_ds

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'final_QT', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [66]:
train_ds.set_format(type='torch',columns=["input_ids","attention_mask","labels"])
test_ds.set_format(type='torch',columns=["input_ids","attention_mask","labels"])

In [67]:
# load_model
model=AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    problem_type="multi_label_classification"
)
model=model.to("cuda")

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/598M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at NAMAA-Space/AraModernBert-Base-V1.0 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [68]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.array(predictions >= 0.5, dtype=float)

    # Calculate accuracy for each label
    accuracies = (predictions == labels).mean(axis=0)

    # Calculate overall metrics
    accuracy = (predictions == labels).mean()

    return {
        "accuracy": accuracy,
        "average_label_accuracy": accuracies.mean()
    }

In [69]:
# trainer
args=TrainingArguments(
    "arabic_quest_classifier_AraModernBert-Base-V1.0",
    learning_rate=2e-5,
    per_device_eval_batch_size=16,
    per_device_train_batch_size=8,
    num_train_epochs=2,
    save_total_limit=2,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=50,
    logging_steps=1,
    warmup_ratio=0.03,
    #fp16=False,
    #gradient_checkpointing=False,
    #gradient_accumulation_steps=4,
    report_to="wandb",
    run_name="arabic_quest_classifier_AraModernBert-Base-V1.0"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [70]:
trainer=Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics
)

In [71]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Average Label Accuracy
50,0.413400,0.397131,0.804286,0.804286
100,0.449700,0.390280,0.805714,0.805714


TrainOutput(global_step=100, training_loss=0.39951802149415017, metrics={'train_runtime': 39.0701, 'train_samples_per_second': 20.425, 'train_steps_per_second': 2.559, 'total_flos': 67983546742272.0, 'train_loss': 0.39951802149415017, 'epoch': 2.0})

In [59]:
from google.colab import userdata
model.push_to_hub("Arabic-question-classifier",token=userdata.get("hf_wr"))

model.safetensors:   0%|          | 0.00/651M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abdeljalilELmajjodi/Arabic-question-classifier/commit/fd1e5afc60d0e40b5e114f6d9b49a4ef926065c9', commit_message='Upload BertForSequenceClassification', commit_description='', oid='fd1e5afc60d0e40b5e114f6d9b49a4ef926065c9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/abdeljalilELmajjodi/Arabic-question-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='abdeljalilELmajjodi/Arabic-question-classifier'), pr_revision=None, pr_num=None)

In [60]:
tokenizer.push_to_hub("Arabic-question-classifier",token=userdata.get("hf_wr"))

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abdeljalilELmajjodi/Arabic-question-classifier/commit/4e427b49fcefae848c53b253bbf0f7858da64ba3', commit_message='Upload tokenizer', commit_description='', oid='4e427b49fcefae848c53b253bbf0f7858da64ba3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/abdeljalilELmajjodi/Arabic-question-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='abdeljalilELmajjodi/Arabic-question-classifier'), pr_revision=None, pr_num=None)

In [72]:
trainer.evaluate()

{'eval_loss': 0.3902799189090729,
 'eval_accuracy': 0.8057142857142857,
 'eval_average_label_accuracy': 0.8057142857142858,
 'eval_runtime': 1.0209,
 'eval_samples_per_second': 97.956,
 'eval_steps_per_second': 6.857,
 'epoch': 2.0}

In [90]:
text="ماهو افضل دواء منوم وذاا تأثير سريع وقوي لاني اعاني من ارق ولا استطيع النوم ابدااا الرجاء الاجابه؟؟"
ids=tokenizer(text,return_tensors="pt", padding=True, truncation=True, max_length=128).to("cuda")
ids

{'input_ids': tensor([[    3,   351,   432, 13633, 17993,   316,   670, 20778,   273,  5079,
         11982, 46856,   413,   703,  1748,   575,   316,   274,   808,   591,
         49922,  6800, 31653, 15795, 17817,  7217,  2402,  6383,     4]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]], device='cuda:0')}

In [109]:
predictions=model(**ids)
predictions=torch.sigmoid(predictions.logits).detach().to("cpu").numpy()[0]
predictons=[labels[i] for i,pred in enumerate(predictions) if pred >= 0.5]
predictons

['B']

In [ ]:
#push to hf
trainer.push_to_hub("model_name",token="")

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/abdeljalilELmajjodi/emotion_classifier/commit/c38c1e540d546e4e9912c068465d744426f25c27', commit_message='emotion-classifier', commit_description='', oid='c38c1e540d546e4e9912c068465d744426f25c27', pr_url=None, repo_url=RepoUrl('https://huggingface.co/abdeljalilELmajjodi/emotion_classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='abdeljalilELmajjodi/emotion_classifier'), pr_revision=None, pr_num=None)